In [21]:
# data_ingestion_gcs_dag.py

import os
import logging
import pandas as pd
from airflow import DAG
from airflow.utils.dates import days_ago

# Import our operators
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

# We installed the following from the requirements file which was specified in the Dockerfile.

# Helps convert our data to parquet format
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pyarrow as pa

In [25]:
    dataset_url_init = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet"
    df = pd.read_parquet(dataset_url_init)
    dataset_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet"
    df_2 = pd.read_parquet(dataset_url)

In [26]:
df = pd.concat([df,df_2])
table = pa.Table.from_pandas(df)

In [28]:
pq.write_table(table, 'example.parquet')

In [29]:
# Specify our dataset
month_list = ['02','03','04','05','06','07','08','09','10','11','12']
# Store environmental variables (in your docker container) locally. The second argument of each `.get` is what it will default to if it's empty.
path_to_local_home = os.environ.get("AIRFLOW_HOME", "/opt/airflow/")
BIGQUERY_DATASET = os.environ.get("BIGQUERY_DATASET", 'trips_green_taxi_data')
dataset_file = "green_taxi_2022"
# Replace CSV with Parquet on our file
parquet_file = dataset_file.replace('.csv', '.parquet')

def load_transform_parquet(src_file):
    dataset_url_init = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet"
    df = pd.read_parquet(dataset_url_init)
    for month in month_list:
        dataset_url =  f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-{month}.parquet"
        df_2 = pd.read_parquet(dataset_url)
        df = pd.concat([df,df_2])
    table = pa.Table.from_pandas(df)
    pq.write_table(table, "green_taxi_2022.parquet")

In [31]:

x = os.environ.get("AIRFLOW_HOME", "/opt/airflow/")
y = "green_taxi_2022"
load_transform_parquet(f"{x}/{y}")
